In [1]:
import pandas as pd, numpy as np
import glob

In [36]:
def fileListLoad(path : str) :
    lst = []
    fileList = glob.glob(path)

    for f in fileList :
        df = pd.read_sas(f)
        lst.append(df)

    return lst

In [37]:
df_main = fileListLoad('Data/all/*.sas7bdat')

In [38]:
df_2019 = df_main[0].copy()
df_2020 = df_main[1].copy()
df_2021 = df_main[2].copy()

class fileLoad() :
    def __init__() :

    

In [39]:
df_sup = fileListLoad('Data/sup/*.sas7bdat')

In [78]:
df_2019_sup = df_sup[0].copy()
df_2020_sup = df_sup[1].copy()
df_2021_sup = df_sup[2].copy()

df_24rc = fileListLoad('Data/24rc/*.sas7bdat')

df_2019_24rc = df_24rc[0].copy()
df_2020_24rc = df_24rc[1].copy()
df_2021_24rc = df_24rc[2].copy()

In [41]:
df_2021_sup.columns

Index(['mod_d', 'ID', 'ID_fam', 'year', 'region', 'town_t', 'apt_t', 'psu',
       'sex', 'age', 'age_month', 'incm', 'ho_incm', 'incm5', 'ho_incm5',
       'edu', 'occp', 'wt_hs', 'wt_itvex', 'wt_oe', 'wt_ex1', 'wt_ige',
       'wt_pft', 'wt_ntr', 'wt_tot', 'wt_oent', 'wt_pfnt', 'wt_ex1nt',
       'wt_igent', 'kstrata', 'LS_SUP', 'LS_KIND', 'LS_CODE', 'LS_NAME',
       'LS_CNAME', 'LS_DUR', 'LS_FQ', 'LS_FQ_U', 'LS_SUP_1D', 'LS_AM',
       'LS_AM_U', 'LS_AM_OU', 'LS_CA', 'LS_PHOS', 'LS_FE', 'LS_VA', 'LS_B1',
       'LS_B2', 'LS_NIAC', 'LS_VITC'],
      dtype='object')

In [42]:
df_2021_sup.iloc[0,:]

mod_d                              b'2023.12.07.'
ID                                  b'A751220401'
ID_fam                                b'A7512204'
year                                       2019.0
region                                        1.0
town_t                                        1.0
apt_t                                         1.0
psu                                       b'A751'
sex                                           1.0
age                                          53.0
age_month                                     NaN
incm                                          1.0
ho_incm                                       1.0
incm5                                         1.0
ho_incm5                                      2.0
edu                                           3.0
occp                                          7.0
wt_hs                                 3605.805922
wt_itvex                              5511.539481
wt_oe                                 7415.114768


In [43]:
df_2021_sup.iloc[1,:]

mod_d                                           b'2023.12.07.'
ID                                               b'A751220401'
ID_fam                                             b'A7512204'
year                                                    2019.0
region                                                     1.0
town_t                                                     1.0
apt_t                                                      1.0
psu                                                    b'A751'
sex                                                        1.0
age                                                       53.0
age_month                                                  NaN
incm                                                       1.0
ho_incm                                                    1.0
incm5                                                      1.0
ho_incm5                                                   2.0
edu                                                    

영양제 column은 

In [44]:
df_2021.shape == df_2021_sup.shape

False

`LS_KIND`가 다른 등 이유로 인해, 해당 dataset의 key에 해당하는 `ID`가 영양 보충제 부분에서 2개 이상 로우에 존재하는 등 문제가 발생하는 자료이다. 영양 보충제 섭취에 관한 자료를 main dataframe에 join하기 위해 정보를 key별로 요약해야 했는데, 각 key별로 하나의 row로 여러 보충제 정보를 얻기 위해 다음의 절차를 따랐다.

* `LS_SUP` : 식이보충제 복용 여부 더미이므로 보존한다. ID별로 다 더하고 더한 개수로 나누면 될 것이다.
* `LS_KIND` : 하나의 row로 줄일 것이므로 제거한다. 
* `LS_CODE`, `LS_NAME`, `LS_AM_OU`, `LS_AM_U`, `LS_AM`, `LS_SUP_ID`, `LS_FQ_U` : 식이보충제 자체에 대한 정보를 담고 있는 값이고 사용하기 어려우므로 드랍한다.

* `LS_DUR`, `LS_FQ` : maximum의 값만을 보존한다.

* `LS_CA`, `LS_PHOS`, `LS_FE`, `LS_VA`, `LS_B1`, `LS_B2`, `LS_NIAC`, `LS_VITC` : 식이보충제 제품별로 영양분이 표시되어 있는 칼럼이므로, 개인별로 해당 칼럼을 모두 더해 사용한다.

In [71]:
sumlist = ['LS_CA', 'LS_PHOS', 'LS_FE', 'LS_VA', 'LS_B1', 'LS_B2', 'LS_NIAC', 'LS_VITC']
droplist = ['mod_d', 'LS_CODE', 'LS_NAME', 'LS_AM_OU', 'LS_AM_U', 'LS_AM', 'LS_SUP_1D', 'LS_FQ_U']

In [59]:
def toOneRow(df) :
    df.drop(columns = droplist, inplace = True)
    gdf = df.groupby('ID')
    for i in sumlist :
        df[i] = gdf[i].transform('sum')
    df['LS_SUP'] = gdf['LS_SUP'].transform('mean')
    df['LS_DUR'], df['LS_PHOS'] = gdf['LS_DUR'].transform('max'), gdf['LS_PHOS'].transform('max')

In [75]:
def toOneRow(df, slist, dlist):
    df.drop(columns = dlist, inplace=True)
    gdf = df.groupby('ID').agg({
        **{col: 'sum' for col in slist},
        'LS_SUP': 'mean',
        'LS_DUR': 'max',
        'LS_PHOS': 'max'
    }).reset_index()
    return gdf

In [79]:
df_2021_fsup = toOneRow(df_2021_sup, sumlist, droplist)

In [81]:
df_2021f = pd.merge(df_2021, df_2021_fsup, how = 'left')

In [83]:
df_2021f.drop(columns = 'mod_d', inplace = True)

In [90]:
df_2021f['LS_SUP'].fillna(0.0, inplace = True)
df_2021f

,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,age_month,...,LS_CA,LS_PHOS,LS_FE,LS_VA,LS_B1,LS_B2,LS_NIAC,LS_VITC,LS_SUP,LS_DUR
0,b'A751215301',b'A7512153',2019.0,1.0,1.0,1.0,b'A751',1.0,61.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,b'A751215303',b'A7512153',2019.0,1.0,1.0,1.0,b'A751',1.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,b'A751220401',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',1.0,53.0,NaN,...,0.00,0.00,0.000,264.000000,0.4000,0.5400,0.000,37.0,1.0,4.0
3,b'A751220402',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',2.0,50.0,NaN,...,75.00,0.00,0.000,1164.000000,8.4000,8.5400,24.000,92.0,1.0,2.0
4,b'A751220403',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',1.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8105,b'R754381902',b'R7543819',2019.0,8.0,1.0,2.0,b'R754',2.0,48.0,NaN,...,0.18,0.16,0.027,0.030833,0.0014,0.0083,0.017,0.9,1.0,5.0
8106,b'R754393801',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',1.0,47.0,NaN,...,0.00,0.00,0.000,0.000000,0.0000,0.0000,0.000,1000.0,1.0,2.0
8107,b'R754393802',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',2.0,43.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
8108,b'R754393803',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',2.0,16.0,NaN,...,0.00,0.00,0.000,0.000000,0.0000,0.0000,0.000,1000.0,1.0,2.0


In [62]:
df_2021_sup['LS_CNAME'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8167 entries, 0 to 8166
Series name: LS_CNAME
Non-Null Count  Dtype 
--------------  ----- 
8167 non-null   object
dtypes: object(1)
memory usage: 63.9+ KB
